In [46]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import FeatureUnion
from sklearn.impute import MissingIndicator
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from os import path
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split

In [21]:
df = pd.read_csv('C:/Users/sunma/Desktop/Intern/house ads/aggregated_houseads (7).csv')

In [22]:
df.shape

(33000, 5)

In [29]:
df.head()

,Date,Creative,Country,Total impressions,Ad server clicks,terminal,position,type,size,Week,total_ctr
0,2023-03-01 03:00:00,20221117_320x50_原创_加拿大新闻,Canada,629,1,None,320x50,原创,320x50,2023-02-27/2023-03-05,0.16
1,2023-03-01 03:00:00,20221117_App&iPad_Image_Live_Top_320x50_东方卫视,Canada,488,0,App&iPad,App&iPad_Image_Live_Top,东方卫视,320x50,2023-02-27/2023-03-05,0.00
2,2023-03-01 03:00:00,20221117_App&iPad_Image_Live_Top_320x50_东方卫视,United States,670,1,App&iPad,App&iPad_Image_Live_Top,东方卫视,320x50,2023-02-27/2023-03-05,0.15
3,2023-03-01 03:00:00,20221117_App&iPad_Image_Pause_原创_300x250_i生活,Canada,109,0,App&iPad,App&iPad_Image_Pause,原创,300x250,2023-02-27/2023-03-05,0.00
4,2023-03-01 03:00:00,20221117_App&iPad_Image_Pause_原创_300x250_i生活,United States,160,0,App&iPad,App&iPad_Image_Pause,原创,300x250,2023-02-27/2023-03-05,0.00


In [24]:
print(df.columns)


Index(['Date', 'Creative', 'Country', 'Total impressions', 'Ad server clicks'], dtype='object')


In [25]:
def extract_info(row):
    split_data = row['Creative'].split('_')
    row['terminal'] = split_data[0]

    # Extract and join non-Chinese characters for 'position'
    position = [x for x in split_data[1:5] if not re.search(r'[\u4e00-\u9fff]', x)]
    row['position'] = '_'.join(position)

    # Extract Chinese characters
    chinese_characters = re.findall(r'[\u4e00-\u9fff]+', row['Creative'])
    if chinese_characters:
        row['type'] = ' '.join(chinese_characters[:1])
    else:
        row['type'] = 'NA'
    
    # Extract size
    size = re.findall(r'\d+x\d+', row['Creative'])
    if size:
        row['size'] = size[0]
    else:
        row['size'] = 'NA'
        
    return row

# Apply the function to each row
df = df.apply(extract_info, axis=1)

In [26]:
# Convert 'Date' to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Create a 'Week' column
df['Week'] = df['Date'].dt.to_period('W')

In [30]:
weekly_df = df.groupby(['Week', 'position', 'size', 'Country', 'terminal']).agg({
    'total_ctr': 'mean',
    # Include any other columns you want to aggregate
}).reset_index()


In [31]:
weekly_df.head()

,Week,position,size,Country,terminal,total_ctr
0,2023-02-27/2023-03-05,App&iPad_Image_Live_Top,320x50,Canada,App&iPad,0.110000
1,2023-02-27/2023-03-05,App&iPad_Image_Live_Top,320x50,United States,App&iPad,0.066000
2,2023-02-27/2023-03-05,App&iPad_Image_Pause,300x250,Canada,App&iPad,0.855859
3,2023-02-27/2023-03-05,App&iPad_Image_Pause,300x250,United States,App&iPad,0.285728
4,2023-02-27/2023-03-05,App_Image_Detail,320x50,Canada,App,0.154672


In [33]:
weekly_df.shape

(665, 6)

In [28]:
conditions = [
    df['position'].str.startswith('App&iPad'),
    df['position'].str.startswith('App'),
    df['position'].str.startswith('Web'),
    df['position'].str.startswith('iPad')
]

choices = ['App&iPad', 'App', 'Web', 'iPad']

df['terminal'] = np.select(conditions, choices, default=None)
df['total_ctr'] = (df['Ad server clicks'] / df['Total impressions'] * 100).round(2)

In [34]:
dummy_df = pd.get_dummies(weekly_df, columns=['Country', 'terminal', 'position', 'size'])

In [37]:
dummy_df.head()

,Week,total_ctr,Country_Canada,Country_United States,terminal_App,terminal_App&iPad,terminal_Web,terminal_iPad,position_App&iPad_Image_Live_Top,position_App&iPad_Image_Pause,...,position_iPad_Image_Home,position_iPad_Image_Home_Banner,position_iPad_Image_Original,position_iPad_Image_Points&Detail,size_1024x363,size_160x600,size_300x250,size_320x50,size_335x190,size_728x90
0,2023-02-27/2023-03-05,0.110000,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
1,2023-02-27/2023-03-05,0.066000,0,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
2,2023-02-27/2023-03-05,0.855859,1,0,0,1,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
3,2023-02-27/2023-03-05,0.285728,0,1,0,1,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
4,2023-02-27/2023-03-05,0.154672,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [38]:
print(dummy_df.columns)

Index(['Week', 'total_ctr', 'Country_Canada', 'Country_United States',
       'terminal_App', 'terminal_App&iPad', 'terminal_Web', 'terminal_iPad',
       'position_App&iPad_Image_Live_Top', 'position_App&iPad_Image_Pause',
       'position_App_Image_Detail', 'position_App_Image_Detail&Original&Intro',
       'position_App_Image_Home', 'position_App_Image_Home_Banner',
       'position_App_Image_Intro', 'position_Web_Image_Banner_Left',
       'position_Web_Image_Banner_Right', 'position_Web_Image_Detail&Original',
       'position_Web_Image_Detail&Points', 'position_Web_Image_Home_Center',
       'position_Web_Image_Home_Left', 'position_Web_Image_Home_Right',
       'position_Web_Image_Live_Center', 'position_Web_Image_Live_Left',
       'position_Web_Image_Live_Right', 'position_Web_Image_Pause',
       'position_Web_Image_VOD_Center', 'position_iPad_Image_Detail',
       'position_iPad_Image_Detail&Original', 'position_iPad_Image_Home',
       'position_iPad_Image_Home_Banner', 'po

In [44]:
# Define predictors and target variable
X = dummy_df.drop(['Week', 'total_ctr'], axis=1) 
y = dummy_df['total_ctr']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instantiate the model
model = LinearRegression()

# Perform cross-validation on the training set
scores = cross_val_score(model, X_train, y_train, cv=5)

# The 'scores' variable now contains 5 scores (one for each fold)
print("Cross-validation scores: ", scores)
print("Average cross-validation score: ", scores.mean())

# Train the model on the entire training set
model.fit(X_train, y_train)

# Evaluate the model on the test set
y_pred = model.predict(X_test)
test_mse = mean_squared_error(y_test, y_pred)
print("Test MSE: ", test_mse)

Cross-validation scores:  [ 0.36340433  0.14941493  0.43997913  0.33464229 -0.62496851]
Average cross-validation score:  0.1324944327041689
Test MSE:  4.683861810912537


In [47]:
#5 different model testing
# Define predictors and target variable
X = dummy_df.drop(['Week', 'total_ctr'], axis=1) 
y = dummy_df['total_ctr']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define models
models = [
    ("Linear Regression", LinearRegression()),
    ("Decision Tree", DecisionTreeRegressor(random_state=42)),
    ("Random Forest", RandomForestRegressor(n_estimators=100, random_state=42)),
    ("Gradient Boosting", GradientBoostingRegressor(random_state=42)),
    ("SVR", SVR())
]

# Evaluate each model
for name, model in models:
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred = model.predict(X_test)
    
    # Compute R^2 score and MSE
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    
    # Print results
    print(f"{name} - R^2 score: {r2:.2f}, MSE: {mse:.2f}")

Linear Regression - R^2 score: 0.36, MSE: 4.68
Decision Tree - R^2 score: 0.52, MSE: 3.49
Random Forest - R^2 score: 0.51, MSE: 3.57
Gradient Boosting - R^2 score: 0.52, MSE: 3.48
SVR - R^2 score: 0.36, MSE: 4.68


In [52]:
#Select the decision tree
decision_model = DecisionTreeRegressor(random_state=42)
decision_model.fit(X_train, y_train)

DecisionTreeRegressor(random_state=42)

In [53]:
import pickle
with open('decision_tree.pkl', 'wb') as file:
    pickle.dump(decision_model, file)

In [50]:
feature_names = X_train.columns
print("Feature names:", feature_names)

Feature names: Index(['Country_Canada', 'Country_United States', 'terminal_App',
       'terminal_App&iPad', 'terminal_Web', 'terminal_iPad',
       'position_App&iPad_Image_Live_Top', 'position_App&iPad_Image_Pause',
       'position_App_Image_Detail', 'position_App_Image_Detail&Original&Intro',
       'position_App_Image_Home', 'position_App_Image_Home_Banner',
       'position_App_Image_Intro', 'position_Web_Image_Banner_Left',
       'position_Web_Image_Banner_Right', 'position_Web_Image_Detail&Original',
       'position_Web_Image_Detail&Points', 'position_Web_Image_Home_Center',
       'position_Web_Image_Home_Left', 'position_Web_Image_Home_Right',
       'position_Web_Image_Live_Center', 'position_Web_Image_Live_Left',
       'position_Web_Image_Live_Right', 'position_Web_Image_Pause',
       'position_Web_Image_VOD_Center', 'position_iPad_Image_Detail',
       'position_iPad_Image_Detail&Original', 'position_iPad_Image_Home',
       'position_iPad_Image_Home_Banner', 'position